## Currency Conversion Tool

https://www.exchangerate-api.com/

In [46]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
import os
from dotenv import load_dotenv
from langchain_core.tools import InjectedToolArg
from typing import Annotated
load_dotenv()


@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    Get the conversion factor from base_currency to target_currency.
    """
    url = f"https://v6.exchangerate-api.com/v6/{os.environ['EXCHANGERATE_API_KEY']}/pair/{base_currency}/{target_currency}"

    response = requests.get(url)
    if response.status_code != 200:
        raise ValueError(f"Error fetching conversion rate: {response.text}")
    
    data = response.json()
    if 'conversion_rate' not in data:
        raise ValueError("Conversion rate not found in response.")
    
    return response.json()



@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    Convert an amount from base_currency to target_currency using the conversion rate.
    """
    return base_currency_value * conversion_rate

In [47]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [48]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'PKR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1748908802,
 'time_last_update_utc': 'Tue, 03 Jun 2025 00:00:02 +0000',
 'time_next_update_unix': 1748995202,
 'time_next_update_utc': 'Wed, 04 Jun 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'PKR',
 'conversion_rate': 281.8059}

In [49]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [50]:
# tool binding
llm = ChatOpenAI()

In [51]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [53]:
messages = [HumanMessage('What is the conversion factor between pkr and USD, and based on that can you convert 10 pkr to usd')]

In [54]:
messages

[HumanMessage(content='What is the conversion factor between pkr and USD, and based on that can you convert 10 pkr to usd', additional_kwargs={}, response_metadata={})]

In [55]:
ai_message = llm_with_tools.invoke(messages)

In [56]:
messages.append(ai_message)

In [57]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'PKR', 'target_currency': 'USD'},
  'id': 'call_729bCvIAlSVaIXMylxzbzET3',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_3f2Opim4SiJ6VnJzbWvIiADh',
  'type': 'tool_call'}]

In [58]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [59]:
messages

[HumanMessage(content='What is the conversion factor between pkr and USD, and based on that can you convert 10 pkr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_729bCvIAlSVaIXMylxzbzET3', 'function': {'arguments': '{"base_currency": "PKR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_3f2Opim4SiJ6VnJzbWvIiADh', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 111, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BeNu8bvpWULTdPegy13ZfjwHZ5n2y', 'service_tier': 'defa

In [60]:
llm_with_tools.invoke(messages).content

'The conversion factor between PKR and USD is 0.003549. \n\nTherefore, 10 PKR is equal to 0.03549 USD.'

In [61]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

C:\Users\Blue\AppData\Local\Temp\ipykernel_13784\627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [63]:
# --- Step 6: Run the Agent ---
user_query = "how much 100usd in pkr"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: The user wants to know how much 100 USD is in Pakistani Rupees (PKR).

Action:
```
{
  "action": "get_conversion_factor",
  "action_input": {
    "base_currency": "USD",
    "target_currency": "PKR"
  }
}
```  

Observation: {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1748908802, 'time_last_update_utc': 'Tue, 03 Jun 2025 00:00:02 +0000', 'time_next_update_unix': 1748995202, 'time_next_update_utc': 'Wed, 04 Jun 2025 00:00:02 +0000', 'base_code': 'USD', 'target_code': 'PKR', 'conversion_rate': 281.8059}
Thought:Action:
```
{
  "action": "convert",
  "action_input": {
    "base_currency_value": 100,
    "conversion_rate": 281.8059
  }
}
```
Observation: 28180.59
Thought:{
  "action": "Final Answer",
  "action_input": "100 USD is approximately 28,180.59 PKR"
}

> Finished chain.
